CHỦ ĐỀ NGHIÊN CỨU: MẠNG HỌC SÂU XỬ LÝ DATASET CÂY LÚA NƯỚC
SỬ DỤNG CÁC MẠNG HỌC SÂU, HỌC TĂNG CƯỜNG ĐỂ CẢI THIỆN ẢNH VÀ PHÁT HIỆN BỆNH CÂY LÚA NƯỚC




In [1]:
# 1. IMPORT THƯ VIỆN CẦN THIẾT
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# 2. CHUẨN BỊ DỮ LIỆU: CHIA DATASET THÀNH TRAIN, VAL, TEST VÀ THỐNG KÊ
original_data_dir = r'F:\tailieu\HỌC_MÁY\datasetriceleaf'  # Thư mục gốc chứa ảnh theo lớp
base_dir = 'rice_data_split'  # Thư mục mới để chứa dữ liệu chia sẵn
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

class_counts = {}  # Lưu số lượng ảnh theo lớp trước chia tách

# Liệt kê tên các lớp và đếm số lượng ảnh
class_names = []
for cls in os.listdir(original_data_dir):
    cls_path = os.path.join(original_data_dir, cls)
    if os.path.isdir(cls_path):
        imgs = os.listdir(cls_path)
        class_counts[cls] = len(imgs)
        class_names.append(cls)

print("Các lớp trong tập dữ liệu:")
for cls in sorted(class_names):
    print(f"- {cls}")

#Xóa thư mục chia cũ
if os.path.exists(base_dir):
    print("Đang xóa thư mục chia cũ để chia lại...")
    shutil.rmtree(base_dir)

# Tạo lại thư mục chia dữ liệu
os.makedirs(train_dir)
os.makedirs(val_dir)
os.makedirs(test_dir)


#CHIA DỮ LIỆU
for cls in class_counts.keys():
    imgs = os.listdir(os.path.join(original_data_dir, cls))
    random.shuffle(imgs)
    n_total = len(imgs)
    n_train = int(n_total * 0.7)
    n_val = int(n_total * 0.2)

    for i, img in enumerate(imgs):
        src = os.path.join(original_data_dir, cls, img)
        if i < n_train:
            dst = os.path.join(train_dir, cls)
        elif i < n_train + n_val:
            dst = os.path.join(val_dir, cls)
        else:
            dst = os.path.join(test_dir, cls)
        os.makedirs(dst, exist_ok=True)
        shutil.copy(src, dst)

#THỐNG KÊ SỐ LƯỢNG ẢNH SAU KHI CHIA
for split in ['train', 'val', 'test']:
    print(f"\nThống kê số ảnh trong tập {split}:")
    split_path = os.path.join(base_dir, split)
    if os.path.exists(split_path):
        for cls in os.listdir(split_path):
            cls_path = os.path.join(split_path, cls)
            if os.path.isdir(cls_path):
                n = len(os.listdir(cls_path))
                print(f"- {cls}: {n} ảnh")
    else:
        print(f"Thư mục {split_path} không tồn tại.")

#KIỂM TRA THƯ MỤC ---
print("\nKiểm tra tồn tại thư mục:")
print("base_dir tồn tại:", os.path.exists(base_dir))
print("Train dir tồn tại:", os.path.exists(train_dir))
print("Val dir tồn tại:", os.path.exists(val_dir))
print("Test dir tồn tại:", os.path.exists(test_dir))

# VẼ BIỂU ĐỒ PHÂN BỐ SỐ LƯỢNG ẢNH BAN ĐẦU THEO LỚP
plt.figure(figsize=(10, 5))
sns.barplot(x=list(class_counts.keys()), y=list(class_counts.values()))
plt.title('Số lượng ảnh theo lớp (trước khi chia)')
plt.xlabel('Lớp bệnh')
plt.ylabel('Số lượng ảnh')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()